In [94]:
import os
import cv2
import numpy as np
from random import choice, randint
import shutil

### Balancing the Fer2013 dataset through data augmentation
We balance the Fer2013 dataset by removing or adding data inside a respective category to reach about 5000 images for each.

In [95]:
# Get a list of all folders inside the folder Data/Fer2013
folders = os.listdir('Fer2013/train')
image_counts = []

# Iterate over each folder
for folder in folders:
    # Count the number of images in each folder
    count = len(os.listdir('Fer2013/train/' + folder))
    image_counts.append(5000-count)

    print('Folder:', folder, ' ---- Count:', count , ' ----- Adding:' , 5000-count)

Folder: angry  ---- Count: 3995  ----- Adding: 1005
Folder: disgust  ---- Count: 436  ----- Adding: 4564
Folder: fear  ---- Count: 4097  ----- Adding: 903
Folder: happy  ---- Count: 7215  ----- Adding: -2215
Folder: neutral  ---- Count: 4965  ----- Adding: 35
Folder: sad  ---- Count: 4830  ----- Adding: 170
Folder: surprise  ---- Count: 3171  ----- Adding: 1829


In [96]:
# Create a new folder BalancedFer2013
if not os.path.exists('BalancedFer2013'):
    os.mkdir('BalancedFer2013')
    
    for dataset in ['train', 'test']:
        os.mkdir(f'BalancedFer2013/{dataset}')
        
        for category in folders:
            os.mkdir(f'BalancedFer2013/{dataset}/{category}')

In [97]:
print(image_counts)

[1005, 4564, 903, -2215, 35, 170, 1829]


#### Augmenting Angry, Fear, Neutral, Sad, Surprise

In [98]:
# Remove the 2nd and 4th count for the disgust and happy images
image_counts = [count for count in image_counts if count not in [4564, -2215]]

# Create new images by rotating the original images for a random amount of degrees between -30 and 30 excluding 0
degrees = list(range(-30,31))
degrees = [degree for degree in degrees if degree != 0]

for i, category in enumerate([folder for folder in folders if folder not in ['disgust', 'happy']]):
    
    source_images = np.random.choice(os.listdir(f'Fer2013/train/{category}'), image_counts[i], replace=False)
    
    for image in source_images:
        img = cv2.imread(f'Fer2013/train/{category}/' + image)
        rows, cols, _ = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), choice(degrees), 1)
        dst = cv2.warpAffine(img, M, (cols, rows))
        cv2.imwrite(f'BalancedFer2013/train/{category}/' + image[:-4] + '_rotated.jpg', dst)
        
    # Copy all the images from the original Fer2013 to the BalancedFer2013 dataset and also add copy the test images
    for dataset in ['test', 'train']:
        source_path = f'Fer2013/{dataset}/{category}/'
        destination_path = f'BalancedFer2013/{dataset}/{category}/'
        
        images = os.listdir(source_path)
        
        for image in images:
            shutil.copyfile(source_path + image, destination_path + image)

#### Reducing Happy

In [99]:
# Copy 5000 random happy images from Fer2013 to BalancedFer2013
happy_images = np.random.choice(os.listdir('Fer2013/train/happy'), 5000, replace=False)

source_path = 'Fer2013/train/happy/'
destination_path = 'BalancedFer2013/train/happy/'

for image in happy_images:
    shutil.copyfile(source_path + image, destination_path + image)

source_path = 'Fer2013/test/happy/'
destination_path = 'BalancedFer2013/test/happy/'    
    
# Copy the test images
for image in os.listdir('Fer2013/test/happy'):
    shutil.copyfile(source_path + image, destination_path + image)

#### Augmenting Disgust

In [100]:
# Augment the disgusting images by rotating each image 11 times by a random degree
rotation_range = list(range(-42, 42))
rotation_range_split = [rotation_range[i:i+7] for i in range(0, len(rotation_range), 7)]

for i in range(int(4564/436)+2):
    
    for image in os.listdir('Fer2013/train/disgust'):
        img = cv2.imread('Fer2013/train/disgust/' + image)
        rows, cols, _ = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), choice(rotation_range_split[i]), 1)
        dst = cv2.warpAffine(img, M, (cols, rows))
        cv2.imwrite(f'BalancedFer2013/train/disgust/' + image[:-4] + '_rotated_' + str(i) + '.jpg', dst)

print(len(os.listdir('BalancedFer2013/train/disgust')))

5232


In [101]:
# Remove 232 random images
images = os.listdir('BalancedFer2013/train/disgust')
remove = []

for i in range(232):
    remove.append(images.pop(randint(0, len(images) - 1)))
    
for image in remove:
    os.remove('BalancedFer2013/train/disgust/' + image)

In [102]:
# Copy the test images
source_path = 'Fer2013/test/disgust/'
destination_path = 'BalancedFer2013/test/disgust/'    
    
# Copy the test images
for image in os.listdir('Fer2013/test/disgust'):
    shutil.copyfile(source_path + image, destination_path + image)